In [13]:
import os
import glob
import pandas as pd
import pyndl.io
import pdb

In [2]:
basepath = '/Users/gracer/Desktop/reversal_learning_logfiles'

# Basic structure
| State | NextState | Reward | Action |  

## Where:
* State = which logo is the reward. There are two possible states
** S1 = Sweet logo is correct
** S2 = Unsweet logo is correct
* Next_state = which state is next, this is dependent on the number of correct
* Reward = Was the action rewarded
** Yes = 1
** No = 2
** Miss = 0
* Action = this is the logo chosen, the participant can choose either the sweetLogo or unsweetLogo or miss

In [3]:
os.path.exists(basepath)

True

In [4]:
data_dict = {'run01':{},'run02':{}}

In [5]:
def find_action_reward(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    X = df[df[1].str.match('Level injecting via pump at address')] 
    Y = X[1].str.split(' ',  expand=True)
    Z = Y[[6,15]]
    Z = Z.rename(columns={6: 'reward', 15: 'Action'})
    return(Z)

Key Press Missed! 

In [6]:
def find_misses(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    X = df[df[1].str.match('Level Key Press Missed')] 
    X['reward'] = 0
    X['Action']= 'miss'
    X = X[['reward','Action']]
    return(X)

In [7]:
def find_correct(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    try:
        X = df[df[1].str.match('Level correct responses')]
        Y = X[1].str.split(' ',  expand=True)
        Z = Y[[3]]
        Z = Z.rename(columns={3: 'corrects'})

        return(Z)
    except KeyError:
        print('no correct responses')
        return('STOP')

In [8]:
def find_flip(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    try:
        X = df[df[1].str.match('Level FLIP')] 
        Y = X[1].str.split(' ',  expand=True)
        Z = Y[[1]]
        Z = Z.rename(columns={1: 'flip'})
        return(Z)
    except KeyError:
        print('no flips')
        return('STOP')
    

Parser breaks on sub-121
The break needed to be a continue and it was in the wrong spot. (Doh!)

In [18]:
data_dict = {'run01':{},'run02':{}}
files = glob.glob(os.path.join(basepath,'*.log'))
files.sort()

for item in files:
    sub = item.split('/')[-1].split('_')[1]
    run = item.split('/')[-1].split('_')[2]
    df1 = find_action_reward(item)
    df2 = find_misses(item)
    df4 = find_flip(item)
    
    
    df_list = [df1, df2,  df4]
    for count, data in enumerate(df_list):
        if isinstance(data, pd.DataFrame) == True:
            print('%s df%s is good'%(sub,count))
        else:
            print('%s df%s is bad kill'%(sub, count))
            data_dict[run][sub] = 'Fail'
    try:
        DF = pd.concat(df_list, axis = 0)
    except TypeError:
        continue
    DF = DF.sort_index()
    
    new = []
    states = ['s1','s2']
    next_state = ['s1']

    test = DF['flip'] 
    new2 = [0]*len(test)
    
    for i,x in enumerate(test):
        if x == 'FLIP':
            states = list(reversed(states))
            new2[i-1] = states[0]
        new.append(states[0])
        new2[i] = states[0]
    DF['states'] = new
    DF['next_states'] = new2
    DF = DF.drop('flip', axis=1)
    DF = DF.dropna()
    data_dict[run][sub] = DF
    outpath = os.path.join(basepath, 'reparsed','sub-%s_%s.csv'%(sub,run))
    DF.to_csv(outpath)

/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

001 df0 is good
001 df1 is good
001 df2 is good
001 df0 is good
001 df1 is good
001 df2 is good
002 df0 is good
002 df1 is good
002 df2 is good
002 df0 is good
002 df1 is good
002 df2 is good
002 df0 is good
002 df1 is good
002 df2 is good
004 df0 is good
004 df1 is good
004 df2 is good
004 df0 is good
004 df1 is good
004 df2 is good
005 df0 is good
005 df1 is good
005 df2 is good
005 df0 is good
005 df1 is good
005 df2 is good
006 df0 is good
006 df1 is good
006 df2 is good
006 df0 is good
006 df1 is good
006 df2 is good
007 df0 is good
007 df1 is good
007 df2 is good
007 df0 is good
007 df1 is good
007 df2 is good
008 df0 is good
008 df1 is good
008 df2 is good
008 df0 is good
008 df1 is good
008 df2 is good


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

009 df0 is good
009 df1 is good
009 df2 is good
009 df0 is good
009 df1 is good
009 df2 is good
010 df0 is good
010 df1 is good
010 df2 is good
010 df0 is good
010 df1 is good
010 df2 is good
011 df0 is good
011 df1 is good
011 df2 is good
011 df0 is good
011 df1 is good
011 df2 is good
013 df0 is good
013 df1 is good
013 df2 is good
013 df0 is good
013 df1 is good
013 df2 is good
014 df0 is good
014 df1 is good
014 df2 is good
014 df0 is good
014 df1 is good
014 df2 is good
015 df0 is good
015 df1 is good
015 df2 is good
016 df0 is good
016 df1 is good
016 df2 is good
016 df0 is good
016 df1 is good
016 df2 is good
017 df0 is good
017 df1 is good
017 df2 is good
017 df0 is good
017 df1 is good
017 df2 is good
019 df0 is good
019 df1 is good
019 df2 is good


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

...

/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

144 df0 is good
144 df1 is good
144 df2 is good
144 df0 is good
144 df1 is good
144 df2 is good
145 df0 is good
145 df1 is good
145 df2 is good
145 df0 is good
145 df1 is good
145 df2 is good
146 df0 is good
146 df1 is good
146 df2 is good
147 df0 is good
147 df1 is good
147 df2 is good
147 df0 is good
147 df1 is good
147 df2 is good
148 df0 is good
148 df1 is good
148 df2 is good
148 df0 is good
148 df1 is good
148 df2 is good
149 df0 is good
149 df1 is good
149 df2 is good
149 df0 is good
149 df1 is good
149 df2 is good
150 df0 is good
150 df1 is good
150 df2 is good
150 df0 is good
150 df1 is good
150 df2 is good
151 df0 is good
151 df1 is good
151 df2 is good
151 df0 is good
151 df1 is good
151 df2 is good
152 df0 is good
152 df1 is good
152 df2 is good


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

152 df0 is good
152 df1 is good
152 df2 is good
153 df0 is good
153 df1 is good
153 df2 is good
153 df0 is good
153 df1 is good
153 df2 is good
154 df0 is good
154 df1 is good
154 df2 is good
154 df0 is good
154 df1 is good
154 df2 is good
156 df0 is good
156 df1 is good
156 df2 is good
156 df0 is good
156 df1 is good
156 df2 is good
157 df0 is good
157 df1 is good
157 df2 is good
157 df0 is good
157 df1 is good
157 df2 is good
159 df0 is good
159 df1 is good
159 df2 is good
159 df0 is good
159 df1 is good
159 df2 is good
160 df0 is good
160 df1 is good
160 df2 is good
160 df0 is good
160 df1 is good
160 df2 is good
160 df0 is good
160 df1 is good
160 df2 is good
161 df0 is good
161 df1 is good
161 df2 is good
no flips
161 df0 is good
161 df1 is good
161 df2 is bad kill
161 df0 is good
161 df1 is good
161 df2 is good
162 df0 is good
162 df1 is good
162 df2 is good
162 df0 is good
162 df1 is good
162 df2 is good
163 df0 is good
163 df1 is good
163 df2 is good
163 df0 is good
163 df1 is 

/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_47925/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [21]:
import pickle

In [22]:
outpath = os.path.join(basepath, 'parsed','RL.pickle')
with open(outpath, 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)